In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import time
import pydde as d

In [2]:
#Parameters
samplenum = 300
epochs = 100
hiddenlayers = [200]
input_size = 3
output_size = 3
learning_rate = 0.01
time_length = 60; #seconds

In [3]:
# Generate simulation
dyn = d.PyDyn('test2.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [4]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [5]:
class Simulate(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input):
        #print(f'input: {input.shape}')
        p = input.clone().numpy().transpose()
        state = dyn.compute(p)
        y_pred = torch.tensor(state.y[-3:])
        #print(f'y_pred: {y_pred.dtype}')
        
        ctx.save_for_backward(input)
        
        return y_pred, input
    
    @staticmethod
    def backward(ctx, grad_output, input):
        #print(grad_output.shape)
        input, = ctx.saved_tensors
        p = input.clone().numpy().transpose()
        state= dyn.compute(p)
        dy_dp = dyn.dy_dp(state, p)
        dy_dp = dy_dp[-3:, :]
        #print(f'shape of dy/dp: {dy_dp.shape}')
        #print(f'shape of grad_output: {grad_output.shape}')
        grad_output = grad_output.unsqueeze(0)
        
        grad_input = torch.tensor(dy_dp).t().mm(grad_output.t()).t()
        return grad_input, None

Simulate = Simulate.apply

## Building the Model

In [6]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        x = self.Relu(x)
        x, p = Simulate(x)
        return x, p


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [7]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()

losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        #print(y_truth.shape)
        #y_truth = y_truth.unsqueeze(0)
        y_pred, p_pred = model(y_truth)
        #print(y_pred.shape)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        #loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        loss = criterion(y_pred.float(), y_truth) # MSE
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')
    i+=1

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

epoch:   0/100  loss: 0.36306608
epoch:   1/100  loss: 0.00168531
epoch:   2/100  loss: 0.00022838
epoch:   3/100  loss: 0.00003377
epoch:   4/100  loss: 0.00006880
epoch:   5/100  loss: 0.00005738
epoch:   6/100  loss: 0.00004872
epoch:   7/100  loss: 0.00038996
epoch:   8/100  loss: 0.00036478
epoch:   9/100  loss: 0.00032820
epoch:  10/100  loss: 0.00023681
epoch:  11/100  loss: 0.00042493
epoch:  12/100  loss: 0.00042731
epoch:  13/100  loss: 0.00052174
epoch:  14/100  loss: 0.00033861
epoch:  15/100  loss: 0.00034452
epoch:  16/100  loss: 0.00035887
epoch:  17/100  loss: 0.00040659
epoch:  18/100  loss: 0.00047475
epoch:  19/100  loss: 0.00037809
epoch:  20/100  loss: 0.00036438
epoch:  21/100  loss: 0.00084546
epoch:  22/100  loss: 0.00057428
epoch:  23/100  loss: 0.00098747
epoch:  24/100  loss: 0.00058393
epoch:  25/100  loss: 0.00058139
epoch:  26/100  loss: 0.00050473
epoch:  27/100  loss: 0.00158632
epoch:  28/100  loss: 0.00064578
epoch:  29/100  loss: 0.00025243
epoch:  30

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved


## Test forward propagation

In [ ]:
p= np.random.rand(180)
p= torch.tensor(p)
y_pred, p_pred = Simulate(p)
y_pred = y_pred.clone().numpy()

yTraj_test = sim.compute(p_pred)

print(y_pred)
print(yTraj_test[-3:])
print(np.sum(yTraj_test[-3:]-y_pred))

## Torch Script Conversion and Saving

In [12]:
input_example = torch.tensor(y_target[0,:])
traced_script_module = torch.jit.trace(model, input_example)

# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
y, output_example = traced_script_module(test_input)
print(output_example[-5:])

# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")


tensor([0.0000, 0.0000, 0.0000, 2.8979, 1.1397], grad_fn=<SliceBackward>)


RuntimeError: 
Could not export Python function call 'Simulate'. Remove calls to Python functions before export. Did you forget add @script or @script_method annotation? If this is a nn.ModuleList, add it to __constants__:
<ipython-input-6-cef5abb0ed37>(22): forward
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/torch/nn/modules/module.py(516): _slow_forward
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/torch/nn/modules/module.py(530): __call__
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/torch/jit/__init__.py(1034): trace_module
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/torch/jit/__init__.py(882): trace
<ipython-input-12-16578e01869c>(2): <module>
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py(3331): run_code
/Users/nicovonau/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py(3254): run_ast_nodes
/Users/nicovonau/Library/Python/3.7/lib/python/site-pa